# LangSmith による評価
評価用のデータはLLMを使って Synthetic Data として利用する。Ragas の合成テストデータはある程度複雑な質問にするための工夫がなされている。


In [1]:
from langchain_community.document_loaders import GitLoader

In [2]:
def file_fileter(file_path:str) -> bool:
    return file_path.endswith('.mdx')

loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_fileter,
)

documents = loader.load()
print(len(documents))

439


In [3]:
# Ragas によるデータ作成
for document in documents:
    document.metadata["filename"]=document.metadata["source"]

In [ ]:
import nest_asyncio
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

nest_asyncio.apply()

generator = TestsetGenerator.from_langchain(
    generator_llm=AzureChatOpenAI(temperature=0,api_version="2024-06-01"),
    critic_llm=AzureChatOpenAI(temperature=0,api_version="2024-06-01"),
    embeddings=AzureOpenAIEmbeddings()
)

testset = generator.generate_with_langchain_docs(
    documents,
    test_size=4,
    distributions={simple:0.5, reasoning: 0.25, multi_context: 0.25}
)

ValidationError: 1 validation error for AzureChatOpenAI
  Value error, Must provide either the `api_version` argument or the `OPENAI_API_VERSION` environment variable [type=value_error, input_value={'temperature': 0, 'model_kwargs': {}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error